In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1 Microsoft.SemanticKernel.Agents.Core, 1.67.1

# Adding functions from a type

In [2]:
using System.ComponentModel;
class KuberenetesSkills
{
    [Description("Runs kubectl commands, just provide the arguments to the kubectl and it will run it and return the result as string")]
    [KernelFunction()]
    public async Task<string> RunKubectl(string kubectlArguements)
    {
        Console.WriteLine($"{kubectlArguements}");
        return "Not implemented";
    }
}

In [3]:
try{
    var x= Microsoft.Extensions.AI.AIJsonUtilities.CreateJsonSchema(typeof(Microsoft.Extensions.AI.AIJsonUtilities), "", false, null, null, null);
    kernel.Plugins.AddFromType<KuberenetesSkills>();}
    catch(Exception ex)
    {
        Console.WriteLine(ex.Message);
        Console.WriteLine(ex.InnerException?.Message);
    }


In [4]:
PromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };

In [5]:
using Microsoft.SemanticKernel.Agents;

var k8sAgent = new ChatCompletionAgent
{
    Name = "KubernetesAgent",
    Instructions = 
        """
        You are an expert in Kubernetes.
        Answer questions about Kubernetes ONLY!.
        """,
    Description = "An expert in Kubernetes",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [6]:

var response = k8sAgent.InvokeAsync("run a new pod in my k8s cluster from image 'myimage_name:latest'");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

run mypod --image=myimage_name:latest
It seems the specific command execution isn't currently implemented. However, you can manually run the following command in your Kubernetes environment:

```bash
kubectl run mypod --image=myimage_name:latest
```

This command will create a new pod named `mypod` using the image `myimage_name:latest`. Let me know if you need further assistance!


# Adding functions from an object

In [7]:
public sealed class TimePlugin
{
    private readonly TimeProvider _timeProvider;

    public TimePlugin(TimeProvider timeProvider)
    {
        this._timeProvider = timeProvider;
    }

    [KernelFunction]
    [Description("Get the date of today Iin UTC as a string")]
    public string GetTodayAsString()
    {
        DateTimeOffset dateTime = this._timeProvider.GetUtcNow();        

        return dateTime.ToString();
    }
}

In [8]:
kernel.ImportPluginFromObject(new TimePlugin(TimeProvider.System));

In [9]:
using Microsoft.SemanticKernel.Agents;

var calendarAgent = new ChatCompletionAgent
{
    Name = "CalendarAgent",
    Instructions = 
        """
        You are an agent that helps with scheduling meeting and manging the user calendar.
        """,
    Description = "An user calendar assistent",
    Kernel = kernel,


    // Instruct SK to allow the function calling
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

In [10]:
var response = k8sAgent.InvokeAsync("what's the date today?");

await foreach (var item in response)
{
    Console.WriteLine(item.Message);
}

Today's date is December 8, 2025.
